In [1]:
import pandas as pd
import numpy as np
np.random.seed(4869)

In [2]:
listing = pd.read_csv('listing.csv')
trans = pd.read_csv('roofstock_transaction_history.csv')
offers = pd.read_csv('roofstock_marketplace_offers_full.csv')

/Users/zhemingchen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,9,11,14,17,38,46,49,50,51,52,53,54,57,58,59,63,64,65,66,67,70,71,72,76,77,79,80,81,82,83,84,85,100,101,102,103,104,105,132,141,151,168,169,170) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
for_sale = listing[listing['LISTING_STATUS']=='For Sale']
for_sale = listing[listing['LIST_PRICE'].notnull()]

In [19]:
for_sale = listing[listing['LISTING_STATUS']=='For Sale']
for_sale = for_sale[for_sale['LIST_PRICE'].notnull()]
for_sale.shape

(24589, 172)

In [20]:
for_sale = for_sale.groupby(['LISTING_ID','LISTING_PUBLISH_TS']).last().reset_index()

In [22]:
for_sale = for_sale[for_sale['LIST_PRICE'].notnull()]
for_sale['START_DATE'] = pd.to_datetime(for_sale['LISTING_PUBLISH_TS'])

In [24]:
for_sale.shape

(5750, 173)

In [ ]:
### OFFERS & TRANS

In [26]:
offers = offers[['OFFER_ITERATION_SUBMIT_DATE','LISTING_ID','OFFER_PRICE','OFFER_DIRECTION','OFFER_STATUS']]
offers['START_DATE'] = 0
offers['LIST_PRICE'] = 0
offers.shape

(31852, 7)

In [31]:
offers['OFFER_ITERATION_SUBMIT_DATE'] = pd.to_datetime(offers['OFFER_ITERATION_SUBMIT_DATE'])

In [32]:
for i in range(0,len(offers)):
    df = for_sale[for_sale['LISTING_ID']==offers.loc[i,'LISTING_ID']]
    df = df.reset_index()
    for j in range(0,len(df)):
        if df.loc[j,'START_DATE'] <= offers.loc[i,'OFFER_ITERATION_SUBMIT_DATE']:
            offers.loc[i,'LIST_PRICE'] = df.loc[j,'LIST_PRICE']
            offers.loc[i,'START_DATE'] = df.loc[j,'START_DATE']
            break
        else:
            pass
offers.shape

(31852, 7)

In [33]:
offers['diff'] = (offers['OFFER_PRICE'] - offers['LIST_PRICE'])/offers['LIST_PRICE'] *100
good_offer = offers[offers['diff']>=-10]
good_offer_1 = good_offer[good_offer['OFFER_DIRECTION']=='BuyerToSeller']
good_offer_2 = good_offer[(good_offer['OFFER_DIRECTION']=='SellerToBuyer') & (good_offer['OFFER_STATUS'] == 'Accepted')]
good_offer = pd.concat([good_offer_1,good_offer_2]) 
good_offer.shape

(15812, 8)

In [34]:
# Transaction
trans = trans[trans['OFFER_ID']==-99]
trans = trans[['TRANSACTION_CREATED_DATE','LISTING_ID','SALE_PRICE']]
trans['START_DATE'] = 0
trans['LIST_PRICE'] = 0
trans['TRANSACTION_CREATED_DATE'] = pd.to_datetime(trans['TRANSACTION_CREATED_DATE'])
trans = trans.reset_index()
trans.shape

(2258, 6)

In [37]:
trans['TRANSACTION_CREATED_DATE'] = pd.to_datetime(trans['TRANSACTION_CREATED_DATE']).apply(lambda x: x.tz_localize('UTC')) 

In [39]:
good_trans = trans[['TRANSACTION_CREATED_DATE','LISTING_ID','SALE_PRICE','START_DATE','LIST_PRICE']]
good_trans.shape

(2258, 5)

In [40]:
main_offer = good_offer[['OFFER_ITERATION_SUBMIT_DATE','LISTING_ID','LIST_PRICE','OFFER_PRICE','START_DATE']]
main_offer = main_offer.rename(columns={'OFFER_ITERATION_SUBMIT_DATE':'END_DATE'})
main_offer['END_DATE'] = pd.to_datetime(main_offer['END_DATE'])
main_offer.shape

(15812, 5)

In [41]:
main_trans = good_trans[['TRANSACTION_CREATED_DATE','LISTING_ID','LIST_PRICE','SALE_PRICE','START_DATE']]
main_trans = main_trans.rename(columns={'TRANSACTION_CREATED_DATE':'END_DATE',
                                       'SALE_PRICE':'OFFER_PRICE'})
main_trans['END_DATE'] = pd.to_datetime(main_trans['END_DATE'])
main_trans.shape

(2258, 5)

In [42]:
main = pd.concat([main_trans,main_offer])
main.shape

(18070, 5)

In [45]:
main = main[main['LIST_PRICE'].notnull()]
main.shape

(18070, 5)

In [ ]:
## MERGE THEM!!

In [47]:
main = main.merge(for_sale, on = ['LISTING_ID','START_DATE'], how = 'left')

ValueError: You are trying to merge on object and datetime64[ns, UTC] columns. If you wish to proceed you should use pd.concat